# About
In notebook 04, we've found that time shifts are different for different type of trucks.
So, in this notebook i'll try to detect time shifts for different trucks and routes

In [1]:
import os
import numpy as np
import pandas as pd
import logging
from shapely.geometry import Point
import plotly.express as px
import geopandas as gpd
import plotly.express as px
from multiprocessing import Pool
from scipy.stats import zscore
from GPSOdyssey import Polaris, Kepler, Void, Vega
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Time shifts

* RFID: by default client provides time shifts in +08:00
* GPS: Different for different trucks

In [2]:
def generate_trip_id(gps):
    gps['trip_id'] = gps['date'].astype('str') + ' ' + gps['vehicle_id']
    return gps

def generate_point_id(gps):
    gps['point_id'] = gps['datetime'].astype('str') + ' ' + gps['vehicle_id']
    return gps

## Data

### Map matching report

In [3]:
map_matching_report = pd.read_csv('/Data/Intermediate/MapMatchingReports/MapMatching_october2020.csv')

map_matching_report = Polaris(map_matching_report) \
    .pandas_df_operation(func_name='rename', arguments={'columns': {'truck_id': 'vehicle_id'}}).df

map_matching_report['vehicle_id'] = map_matching_report['vehicle_id'].str.replace(' ', '-')

### 1. Route vehicle assignment

In [4]:
I_ROUTE_VEHICLE_ASSIGNMENT = '/Data/Intermediate/assignments/route_vehicle_assignments_october.csv'

route_vehic_assign = pd.read_csv(I_ROUTE_VEHICLE_ASSIGNMENT)
route_vehic_assign = route_vehic_assign[route_vehic_assign['route_type'] == 'JS-SE'] \
    .groupby(['route_type']).agg({'vehicle_id': 'unique'}).reset_index()

route_vehic_assign = route_vehic_assign.explode('vehicle_id')

### 2. RFID

In [14]:
S_RFID_OCTOBER = '/Data/Source/RFID/RFID_october.csv'


rfid = pd.read_csv(S_RFID_OCTOBER)
rfid['is_rfid_active'] = True

RFID_COLUMNS = ['date', 'time', 'unixtime', 'vehicle_id',
                'RFID', 'is_rfid_active', 'Latitude', 'Longitude']
rfid = rfid.loc[:, RFID_COLUMNS]
rfid.columns = [str(x).lower() for x in rfid.columns]

rfid = rfid.sort_values(by='unixtime')
rfid['datetime'] = rfid['date'] + ' ' + rfid['time']


rfid = generate_trip_id(rfid)
rfid = generate_point_id(rfid)

,date,time,unixtime,vehicle_id,rfid,is_rfid_active,latitude,longitude,datetime,trip_id,point_id
25169,2020-10-01,06:38:42,1.601534e+09,XE-5630M,E2000019950C00710980BB82,True,1.338714,103.776499,2020-10-01 06:38:42,2020-10-01 XE-5630M,2020-10-01 06:38:42 XE-5630M
25170,2020-10-01,06:39:56,1.601534e+09,XE-5630M,E2000019950C01380990B9EC,True,1.338814,103.776459,2020-10-01 06:39:56,2020-10-01 XE-5630M,2020-10-01 06:39:56 XE-5630M
25171,2020-10-01,06:40:10,1.601534e+09,XE-5630M,E2000019951001870580E1AB,True,1.338790,103.776400,2020-10-01 06:40:10,2020-10-01 XE-5630M,2020-10-01 06:40:10 XE-5630M
25172,2020-10-01,06:40:33,1.601534e+09,XE-5630M,E2000019951001560640DAB7,True,1.338797,103.776421,2020-10-01 06:40:33,2020-10-01 XE-5630M,2020-10-01 06:40:33 XE-5630M
25173,2020-10-01,06:41:01,1.601534e+09,XE-5630M,E2000019951001460720D37D,True,1.338787,103.776409,2020-10-01 06:41:01,2020-10-01 XE-5630M,2020-10-01 06:41:01 XE-5630M
...,...,...,...,...,...,...,...,...,...,...,...
40166,2020-10-31,18:01:16,1.604167e+09,XE-5640J,E20000199504004719804DB9,True,1.375479,103.687636,2020-10-31 18:01:16,2020-10-31 XE-5640J,2020-10-31 18:01:16 XE-5640J
40167,2020-10-31,18:03:56,1.604167e+09,XE-5640J,E2000019950401312330280E,True,1.372425,103.688925,2020-10-31 18:03:56,2020-10-31 XE-5640J,2020-10-31 18:03:56 XE-5640J
40168,2020-10-31,18:04:44,1.604167e+09,XE-5640J,E20000199504019624501DDE,True,1.372200,103.689076,2020-10-31 18:04:44,2020-10-31 XE-5640J,2020-10-31 18:04:44 XE-5640J
40169,2020-10-31,18:14:39,1.604168e+09,XE-5640J,E2000019951002130670D964,True,1.372366,103.693915,2020-10-31 18:14:39,2020-10-31 XE-5640J,2020-10-31 18:14:39 XE-5640J


#### Select RFID day with larges amount records

In [6]:
# 1. Select only JS-SE + Count amount of records
rfid_agg = rfid.groupby(['date', 'vehicle_id']).agg(count=('time', 'count')).reset_index()

# 2. Select trucks of JS-SE route with largest amount of gps records
rfid_agg = rfid_agg.merge(rfid_agg.groupby(['vehicle_id']).agg(count=('count', 'max')).reset_index(),
                          how='inner', on=['vehicle_id', 'count'])

# 3. Rmoeve rec vehicle id
rfid_agg = rfid_agg[rfid_agg['vehicle_id'].str.slice(stop=2) == 'XE']

# 3. Select actual rfid
rfid = rfid.merge(rfid_agg[['date', 'vehicle_id']],
                  how='inner', on=['date', 'vehicle_id'])

### 3. GPS

In [7]:
map_matching_report = map_matching_report.merge(rfid_agg.rename(columns={'date': 'waste_collection_date'}), how='inner',
                                                on=['waste_collection_date', 'vehicle_id'])



In [8]:
%%time
S_CLUSTERED_GPS = '/Data/Source/OctoberGPS/'


vega = Vega(engine='pandas')
gps = vega.read_from_dir(directory=S_CLUSTERED_GPS,
                         file_extensions='.csv',
                         args={'parse_dates': ['time']},
                         concatenate=True, amt_in_parallel=6,
                         filenames=map_matching_report['csv_file'])


gps = Polaris(gps) \
    .select_columns(['truck_id', 'lon', 'lat', 'time', 'lon_match', 'lat_match']) \
    .pandas_df_operation(func_name='rename', arguments={'columns': {'truck_id': 'vehicle_id', 'time': 'datetime'}}) \
    .construct_datetime(datetime='datetime', offset='-02:00') \
    .remove_tz(time_col='datetime') \
    .add_date_col('datetime', 'date') \
    .add_time_col('datetime', 'time') \
    .add_unixtime('datetime', 'unixtime') \
    .df

gps = generate_trip_id(gps)
gps = generate_point_id(gps)
gps['unixtime'] = gps['unixtime'].astype('int')

CPU times: user 2.69 s, sys: 71.9 ms, total: 2.76 s
Wall time: 2.83 s


# 4. RFID -> GPS

In [9]:
gps['datetime'] = gps['datetime'].astype('str')
rfid['datetime'] = rfid['datetime'].astype('str')

In [10]:
gps_m = gps.merge(rfid[['rfid', 'vehicle_id', 'datetime', 'unixtime',
                        'is_rfid_active', 'latitude', 'longitude']],
                  on=['vehicle_id', 'datetime'], how='outer')

gps_m['is_overlap'] = gps_m['lat_match'].isna()
gps_m.sort_values(['vehicle_id', 'datetime'], inplace=True)

cols2fill = ['lon', 'lat', 'time', 'vehicle_id', 'date', 'datetime', 'time',
             'unixtime_x', 'lon_match', 'lat_match', 'trip_id', 'point_id']
gps_m[cols2fill] = gps_m[cols2fill].fillna(method='ffill')

gps_m['is_rfid_active'] = ~gps_m['rfid'].isna()

In [11]:
gps_m[['latitude', 'longitude']] = gps_m[['latitude', 'longitude']].astype('float', errors='ignore')

# 4. Visualization

In [12]:
MAP_CONF_PATH = 'MAP_CONFIGS/05_TIME_SHIFT_ANALYSIS.json'

kepler = Kepler(data={'gps': gps_m, 'rfid': rfid}, height=800, config_path=MAP_CONF_PATH)
kepler.render_kepler_map()
kepler.get_rendered_map()

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['rfid'], 'id': 'dj9j3zie', '…

In [ ]:
# kepler.save_map_config(path=MAP_CONF_PATH, overwrite_config=True)

In [ ]:
# kepler.save_map_to_html_local('time_shifts_of_same_vehicle.html', overwrite_html=True)

# 5. Time shifts

RFID - GPS

In [13]:
route_vehic_assign

,route_type,vehicle_id
0,JS-SE,XE-5638S
0,JS-SE,XE-5577J
0,JS-SE,XE-5629T


In [ ]:
{
    'XE-5638S': '-02:00',
    'XE-5598Z': '-02:00',
    'XE-5628X': '-02:00',
    'XE-5577J': '-02:00',
    'XE-5665M': '-02:00',
    'XE-5629T': '-02:00',
    'XE-5632H': '-02:00',
    'XE-5680T': '-02:00',
    'XE-5559L': '-02:00',
    'XE-5610X': '-02:00',
    'XE-5636Y': '-02:00',
    'XE-5640J': '-02:00',
    'XE-5612R': '-02:00',
    # 'XE-5705G': '-02:00',
    'XE-5609B': '-01:00',
    'XE-5620S': '-01:00',
    'XE-5639P': '-01:00',
    'XE-5748T': '-01:00',
    'XE-5630M': '-01:00',
    # 'XE-5705G': '-01:00',
}

## Summary

At the end we see different time shifts for the same truck at different day.